In [1]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip
from pyspark.sql.functions import *
import ConnectionConfig as cc
from pyspark.sql.types import *
import requests
from datetime import datetime, timedelta

cc.setupEnvironment()

In [2]:
#active session
spark = cc.startLocalCluster("DIM_WEATHER",4)
spark.getActiveSession()

In [3]:
#EXTRACT
cc.set_connectionProfile("VeloBike")
print(cc.create_jdbc())

jdbc:postgresql://localhost:5433/velodb


In [4]:
#creating the weather
weather_dim_data = [Row(weather_type_SK=1, weather_type="Unpleasant", description="Any form of precipitation."), Row(weather_type_SK=2, weather_type="Pleasant", description="Temperature above 15°C and sunny."), Row(weather_type_SK=3, weather_type="Neutral", description="All other conditions."), Row(weather_type_SK=4, weather_type="Unknown", description="Weather data not available.")]


# creating the schema
dim_weather_schema = StructType([StructField("weather_type_SK", IntegerType(), False), StructField("weather_type", StringType(), False), StructField("description", StringType(), False)])

In [5]:
#creating the DataFrame
dim_weather_df = spark.createDataFrame(weather_dim_data, schema=dim_weather_schema)

dim_weather_df.show(truncate=False)

+---------------+------------+---------------------------------+
|weather_type_SK|weather_type|description                      |
+---------------+------------+---------------------------------+
|1              |Unpleasant  |Any form of precipitation.       |
|2              |Pleasant    |Temperature above 15°C and sunny.|
|3              |Neutral     |All other conditions.            |
|4              |Unknown     |Weather data not available.      |
+---------------+------------+---------------------------------+



In [6]:
#adding it as dimention
dim_weather_df.write.format("delta").mode("overwrite").saveAsTable("dimWeather")
dim_weather_df.repartition(1).write.format("parquet").mode("overwrite").saveAsTable("dimWeather_pq")

In [7]:
spark.stop()